In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
# redis-12025.c85.us-east-1-2.ec2.cloud.redislabs.com:12025
hostname='redis-14366.c281.us-east-1-2.ec2.cloud.redislabs.com'
port=14366
password='3zs23egudfH9g21iFS3ebNHcdFjMTF43'
r=redis.StrictRedis(host=hostname,
                    port=port,
                    password=password)

In [3]:
r.ping()

True

In [5]:
# r.keys()

In [6]:
# load numpy zip data
f=np.load('DataFrame_students_teachers.npz',allow_pickle=True)
x_values=f['arr_0']
col=f['arr_1']
df=pd.DataFrame(x_values,columns=col)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ..."
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613..."
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999..."
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597..."
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732..."


In [7]:
# We have to store data in redis in hash table 
'''there cannot be 2 keys for a value therefore we will
combine name and role into 1 column'''

df['name_role']=df['Name']+'@'+df['Role'] 

In [8]:
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.4324317, 1.074975, -0.12851736, -1.833421, ...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94846946, -0.5583677, 0.09491567, -1.279613...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.72002447, -1.4682528, -0.77743137, -1.45999...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48562637, -0.61965686, -0.4072629, 1.068597...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.1912692, -0.48850536, -1.6902622, 0.4422732...",Scarlett Johansson@Student


In [9]:
records=df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

In [10]:
# records

# Save data  in redis hashes

In [11]:
#cannot store facial features directly i.e array we need to conver it to bytes
for record in records:
    name_role=record['name_role']
    vector=record['Facial_Features']
    
    v_bytes=vector.tobytes()
    r.hset(name='academy:register',key=name_role,value=v_bytes)

# Registeration Form

In [25]:
import cv2
from insightface.app import FaceAnalysis

In [37]:
# configure model
faceapp=FaceAnalysis(name='buffalo_sc',
                     root='insightface_model',
                     providers=['CPUExecutionProvider'])
faceapp.prepare(ctx_id=0,det_size=(640,640),det_thresh=0.5)
# don't set threshold below 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


## 1. Collect Person and Role

In [28]:
person=input("Enter your name: ")

trials=3
for i in range(trials):
    role=input("""Please choose:
    1. Student
    2. Teacher
    Enter either 1 or 2.
    """)
    if role=='1':
        role='Student'
        break
    
    elif role=='2':
        role='Teacher'
        break
        
    else:
        print("Invalid Entry")
        if(i==2):
            print("Max limit for trials reached!!")
            
key=person+'@'+role
print(key)


Enter your name: Sam
Please choose:
    1. Student
    2. Teacher
    Enter either 1 or 2.
    1
Sam@Student


In [41]:
# import cv2
# import faceapp  # Assuming you have the faceapp module imported

# define a video capture object
cap = cv2.VideoCapture(0)
face_embeddings = []
samples = 0

while True:
    # Capture the video frame by frame
    ret, frame = vid.read()
    if not ret:
        print("Unable to read camera")
        break

    # get results from insightface
    results = faceapp.get(frame, max_num=1)
    for res in results:
        samples += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0))

        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)

    if samples >= 200:
        break

    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()


In [49]:
# import the opencv library 
import cv2 


# define a video capture object 
vid = cv2.VideoCapture(0) 

while(True): 
	
	# Capture the video frame 
	# by frame 
	ret, frame = vid.read() 

	# Display the resulting frame 
	cv2.imshow('frame', frame) 
	
	# the 'q' button is set as the 
	# quitting button you may use any 
	# desired button of your choice 
	if cv2.waitKey(1) & 0xFF == ord('q'): 
		break

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 


In [43]:
len(face_embeddings)

200

# # Optimise data and store in redis

In [44]:
x_mean=np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [45]:
# convert to bytes
x_mean_bytes=x_mean.tobytes()

In [46]:
key

'Sam@Student'

In [47]:
# save key and bytes to database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1

In [48]:
r.hgetall(name='academy:register')

{b'Morgan Freeman@Teacher': b'\x05\xa4\xf8>\xd5\xa1\x1e\xbf\xc3\x84\xd0\xbe\xc9\xc7\x88?VN\x0e\xbeA\xd8,\xbd\xd2\xd7\x82?\xfe\x10\x93?\xe6-\xc0>\xf0\x89$\xbe\xf7M\x0b?\x9e)\xcd?\xfbA\x07@\xbb?i\xbf\xe9\xac\xb5?"\x81s?H\x8d\xc6?\x07\xde\x88\xbfeV\x0b\xbe\xe2:\xc5\xbe\x8e\xd5\x9f>\xf1\xdc\xa5\xbe~\xc81>\xfazb?X>&\xbf\x88\xe1\x8f\xbe0cq\xbe\x8f\\\xa2?\x91\x0c\x81\xbf\xe3\xe1\xb1>\x8e\xc2\x19\xbf8c\xe4\xbeN\x1f\xc4?*\xd7\xd8\xbf\xa8|\x89?K\xa3\x0f\xbd\xdee\xf9?\xfd\xf5\xff>e\x06h?\x1d\x9b\xd2?\xc6\xc22\xbf\xd0\xd2\x93?B\xa7\xc5>\n\xc6\x85\xbf\xe0\x95\xa6?\xe2e\xee\xbeY\xc6\x17@\xb3\xed\x83>\xd3\xa7\xf9\xbfO\x82\xb2>\xb4V!\xbfSL\xdf\xbf\xfaR\xc5\xbf\x1d\xa6\x94?\xfeE\x8b\xbf\xa5$l\xbfz\xc1X\xbe\xb4p\x8e?\xfa\xf7>\xbf\x1a@c\xbf\xe6\xf2}\xbf\xb8\x81A\xbf\xc3\xd84\xbe(t5?\xdd`G\xbe\xc1)\x16>x{\x0c?F\xa9h\xbf+\x83\xa8\xbf\x81\xbe\xa1\xbe\xd4\x15\x8f\xbfN^\xc4?\x1e\x8f\x14\xbe\x1dx&\xbf\xb4\x0e)@\xaa\x16\x98\xbe\x17\x14\xc7>\xe6\xbc>?\x1ds\xbd\xbf\xa8\xcc\xa6?\xfa$\xd5\xbf\xa9/)?&\xe2?\xbd\xa6\x